In [ ]:
from langchain.text_splitter import CharacterTextSplitter

pdf_text_splitter = CharacterTextSplitter(chunk_size=1500,chunk_overlap  = 200)
document_sections = pdf_text_splitter.split_documents(documents)
len(document_sections)


In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
query = "Tell me more about the graphic desingers"
qa.run(query)


In [11]:
from langchain.text_splitter import CharacterTextSplitter

pdf_text_splitter = CharacterTextSplitter(chunk_size=1500,chunk_overlap  = 200)
document_sections = pdf_text_splitter.split_documents(documents)
len(document_sections)


2

In [13]:

retriever = db.as_retriever(search_kwargs=dict(k=3))
